### 5.CNN_finetuning_model
#### 1) model summary

In [10]:
!pip install graphviz

In [11]:
import torch
from torchsummary import summary
import torchvision.models as models
import torch.nn as nn
from torchinfo import summary as summary2
from torchviz import make_dot
from torch.autograd import Variable



In [4]:
# model 구조 : AlexNet 뒤에 이진 분류를 위한 layer 추가
device = "cuda" if torch.cuda.is_available() else "cpu"
print('device', device)

model = models.alexnet(pretrained=True)
num_features = model.classifier[6].in_features
print('num_features', num_features)
model.classifier[6] = nn.Linear(num_features, 2)
model = model.to(device)


device cuda


c:\Users\82105\anaconda3\envs\R-CNN\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\82105\anaconda3\envs\R-CNN\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num_features 4096


In [5]:
# train된 weight
chechkpoint_path = './computer_vision/object_detection/c_o_1_rcnn/R-CNN/models/alexnet_car.pth'

checkpoint = torch.load(chechkpoint_path, map_location= device)
model.load_state_dict(checkpoint)


FileNotFoundError: [Errno 2] No such file or directory: './computer_vision/object_detection/c_o_1_rcnn/R-CNN/models/alexnet_car.pth'

In [6]:
input_parameter = next(model.parameters())
input_parameter.size()


torch.Size([64, 3, 11, 11])

In [7]:
summary(model, (3, 256,256))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 63, 63]          23,296
              ReLU-2           [-1, 64, 63, 63]               0
         MaxPool2d-3           [-1, 64, 31, 31]               0
            Conv2d-4          [-1, 192, 31, 31]         307,392
              ReLU-5          [-1, 192, 31, 31]               0
         MaxPool2d-6          [-1, 192, 15, 15]               0
            Conv2d-7          [-1, 384, 15, 15]         663,936
              ReLU-8          [-1, 384, 15, 15]               0
            Conv2d-9          [-1, 256, 15, 15]         884,992
             ReLU-10          [-1, 256, 15, 15]               0
           Conv2d-11          [-1, 256, 15, 15]         590,080
             ReLU-12          [-1, 256, 15, 15]               0
        MaxPool2d-13            [-1, 256, 7, 7]               0
AdaptiveAvgPool2d-14            [-1, 25

In [8]:
# input : [batch, channels, width, height]
summary2(model, (64, 3, 256,256))


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [64, 2]                   --
├─Sequential: 1-1                        [64, 256, 7, 7]           --
│    └─Conv2d: 2-1                       [64, 64, 63, 63]          23,296
│    └─ReLU: 2-2                         [64, 64, 63, 63]          --
│    └─MaxPool2d: 2-3                    [64, 64, 31, 31]          --
│    └─Conv2d: 2-4                       [64, 192, 31, 31]         307,392
│    └─ReLU: 2-5                         [64, 192, 31, 31]         --
│    └─MaxPool2d: 2-6                    [64, 192, 15, 15]         --
│    └─Conv2d: 2-7                       [64, 384, 15, 15]         663,936
│    └─ReLU: 2-8                         [64, 384, 15, 15]         --
│    └─Conv2d: 2-9                       [64, 256, 15, 15]         884,992
│    └─ReLU: 2-10                        [64, 256, 15, 15]         --
│    └─Conv2d: 2-11                      [64, 256, 15, 15]        

In [12]:
#### 2) graph 생성

In [23]:
# Variable을 통하여 Input 생성
x = Variable(torch.randn(256, 256)).to(device)

# 앞에서 생성한 model에 Input을 x로 입력한 뒤 (model(x))  graph.png 로 이미지를 출력합니다.
make_dot(model(x), params=dict(model.named_parameters())).render("graph", format="png")

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [256, 256]

In [5]:
model.keys()

dict_keys(['epoch', 'loss', 'model', 'optimizer'])

In [8]:
print('loss', model['loss'])
print('eposh', model['epoch'])
print('model', model['model'])
print('optimizer', model['optimizer'])


loss 0.5914099882159637
eposh 24
model OrderedDict([('features.0.weight', tensor([[[[ 1.1293e-01,  8.7315e-02,  8.9105e-02,  ...,  5.0998e-02,
            1.4785e-02,  4.3156e-02],
          [ 6.9159e-02,  3.2120e-02,  4.6894e-02,  ...,  2.2586e-02,
           -1.6280e-02, -1.5058e-03],
          [ 6.9529e-02,  3.1528e-02,  4.8854e-02,  ...,  4.1986e-02,
            7.4416e-03,  8.0826e-03],
          ...,
          [ 8.7477e-02,  9.9870e-02,  6.5344e-02,  ..., -2.0104e-01,
           -1.2845e-01, -1.1426e-01],
          [ 3.9327e-02,  6.2301e-02,  3.4125e-02,  ..., -1.9999e-01,
           -1.1231e-01, -1.0801e-01],
          [ 4.3019e-02,  5.9420e-02,  2.2056e-02,  ..., -1.1736e-01,
           -9.5295e-02, -8.5355e-02]],

         [[-7.8989e-02, -6.5583e-02, -8.8368e-02,  ..., -4.4850e-03,
           -3.0905e-02,  1.9993e-02],
          [-7.5106e-02, -7.5012e-02, -8.3655e-02,  ..., -6.3169e-03,
           -3.4426e-02,  6.0303e-03],
          [-1.0586e-01, -9.3569e-02, -1.1262e-01,  ..